# Coin Flippage: How much should the Gambler bet based on his current Capital

In [1]:
# Formulating the Coin Flippage Problem as an MDP ( We asume probabilities of head and tails)
import torch

In [45]:
#1.Specify Discount Factor and convergence threshold, just like in the frozen lake problem
gamma=1
threshold=0.000001

In [46]:
#2.TOTAL_NUMBER OF STATES:In this case, the number of posible states is the total capital the gambler can have which is between $0-$100
capital_max=100
n_state=capital_max+1

In [47]:
#3.REWARDS VECTOR:The goal is to reach $100 in capital therefore if the agent is in s=101 r=1 for the rest is 0
rewards=torch.zeros(n_state)
rewards[-1]=1
rewards

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [48]:
#4.PROBABILITY OF GETTING HEADS:
head_prob=0.4

In [49]:
#5.CREATING AN ENVIROMENT:We use a dictionary
env={'capital_max':capital_max,
     'head_prob':head_prob,
     'rewards':rewards,
     'n_state':n_state}
    

# Value Iteration

In [50]:
def value_iteration(env,gamma,threshold):
    #1.Loading enviroment Variables
    head_prob=env['head_prob']
    n_state=env['n_state']
    capital_max=env['capital_max']
    rewards=env['rewards']
    #2.Create an initial empty Value function to start iterating over
    V=torch.zeros(n_state)
    while True:
        V_temp=torch.zeros(n_state)
    #3.Start iterating over every state. VERY IMPORTANT: To start the loop at state 1 because we can have no 0 in my pocket. Is like saying the game if over so it does not count
        for state in range(1,capital_max):
    #4.The number of Posible actions for state 1 for example , torch.zeros(min(1,100-1)+1) gives me a vector of only 2 posible actions             
            v_actions=torch.zeros(min(state,capital_max-state)+1)
    #5.The vector of posible actions changes depending on the state I am in!!!! Wow!     
            for action in range(1,min(state,capital_max-state)+1):
                    v_actions[action]+=head_prob*(rewards[state+action]+gamma*V[state+action])
                    v_actions[action]+=(1-head_prob)*(rewards[state-action] + gamma*V[state-action])
            V_temp[state]=torch.max(v_actions)
        max_delta=torch.max(torch.abs(V-V_temp))
        V=V_temp
        if max_delta<=threshold:
            break
    return V           

In [51]:
V=value_iteration(env,gamma,threshold)
V

tensor([0.0000, 0.0021, 0.0052, 0.0092, 0.0129, 0.0174, 0.0231, 0.0278, 0.0323,
        0.0377, 0.0435, 0.0504, 0.0577, 0.0652, 0.0695, 0.0744, 0.0807, 0.0866,
        0.0942, 0.1031, 0.1087, 0.1160, 0.1259, 0.1336, 0.1441, 0.1600, 0.1631,
        0.1677, 0.1738, 0.1794, 0.1861, 0.1946, 0.2017, 0.2084, 0.2165, 0.2252,
        0.2355, 0.2465, 0.2579, 0.2643, 0.2716, 0.2810, 0.2899, 0.3013, 0.3147,
        0.3230, 0.3339, 0.3488, 0.3604, 0.3762, 0.4000, 0.4031, 0.4077, 0.4138,
        0.4194, 0.4261, 0.4346, 0.4417, 0.4484, 0.4565, 0.4652, 0.4755, 0.4865,
        0.4979, 0.5043, 0.5116, 0.5210, 0.5299, 0.5413, 0.5547, 0.5630, 0.5739,
        0.5888, 0.6004, 0.6162, 0.6400, 0.6446, 0.6516, 0.6608, 0.6690, 0.6791,
        0.6919, 0.7026, 0.7126, 0.7248, 0.7378, 0.7533, 0.7697, 0.7868, 0.7965,
        0.8075, 0.8215, 0.8349, 0.8520, 0.8721, 0.8845, 0.9009, 0.9232, 0.9406,
        0.9643, 0.0000])

In [52]:
# We can extract the Best policy

In [57]:
def extract_optimal_policy(env, V_optimal, gamma):
    #1.Loading enviroment Variables
    head_prob=env['head_prob']
    n_state=env['n_state']
    capital_max=env['capital_max']
    rewards=env['rewards']
    #2.Create an initial empty policy. It has 101 of length and it depends on the amount of money I have. So what it say if I have fx. $30 USD I should be #XX
    Optimal_Policy=torch.zeros(capital_max)
    #3.Start iterating over every state. VERY IMPORTANT: To start the loop at state 1 because we can have no 0 in my pocket. Is like saying the game if over so it does not count
    for state in range(1,capital_max): 
    #4.The number of Posible actions for state 1 for example , torch.zeros(min(1,100-1)+1) gives me a vector of only 2 posible actions             
        v_actions=torch.zeros(n_state)
    #5.The vector of posible actions changes depending on the state I am in!!!! Wow!     
        for action in range(1,min(state,capital_max-state)+1):
            v_actions[action]+=head_prob*(rewards[state+action]+gamma*V[state+action])
            v_actions[action]+=(1-head_prob)*(rewards[state-action] + gamma*V[state-action])
            print(state , action ,v_actions)
        Optimal_Policy[state]=torch.argmax(v_actions)
    return Optimal_Policy     

In [58]:
extract_optimal_policy(env, V, gamma)

1 1 tensor([0.0000, 0.0021, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])
2 1 tensor([0.0000, 0.0049, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
       

21 16 tensor([0.0000, 0.1155, 0.1153, 0.1142, 0.1160, 0.1137, 0.1118, 0.1113, 0.1109,
        0.1090, 0.1081, 0.1068, 0.1060, 0.1060, 0.1068, 0.1081, 0.1090, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])
21 17 tensor([0.0000, 0.1155, 0.1153, 0.1142, 0.1160, 0.1137, 0.1118, 0.1113, 0.1109,
   

29 28 tensor([0.0000, 0.1787, 0.1785, 0.1785, 0.1794, 0.1731, 0.1702, 0.1697, 0.1682,
        0.1683, 0.1676, 0.1652, 0.1644, 0.1644, 0.1652, 0.1676, 0.1683, 0.1682,
        0.1697, 0.1702, 0.1731, 0.1794, 0.1779, 0.1769, 0.1760, 0.1755, 0.1760,
        0.1769, 0.1779, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])
29 29 tensor([0.0000, 0.1787, 0.1785, 0.1785, 0.1794, 0.1731, 0.1702, 0.1697, 0.1682,
   

36 2 tensor([0.0000, 0.2337, 0.2331, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])
36 3 tensor([0.0000, 0.2337, 0.2331, 0.2308, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
     

41 10 tensor([0.0000, 0.2790, 0.2791, 0.2806, 0.2771, 0.2749, 0.2746, 0.2741, 0.2755,
        0.2810, 0.2780, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])
41 11 tensor([0.0000, 0.2790, 0.2791, 0.2806, 0.2771, 0.2749, 0.2746, 0.2741, 0.2755,
   

...

63 1 tensor([0.0000, 0.4936, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])
63 2 tensor([0.0000, 0.4936, 0.4900, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
     

68 27 tensor([0.0000, 0.5398, 0.5378, 0.5366, 0.5381, 0.5389, 0.5384, 0.5413, 0.5370,
        0.5346, 0.5334, 0.5327, 0.5324, 0.5324, 0.5327, 0.5334, 0.5346, 0.5370,
        0.5413, 0.5336, 0.5309, 0.5279, 0.5234, 0.5224, 0.5228, 0.5216, 0.5228,
        0.5224, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])
68 28 tensor([0.0000, 0.5398, 0.5378, 0.5366, 0.5381, 0.5389, 0.5384, 0.5413, 0.5370,
   

76 19 tensor([0.0000, 0.6446, 0.6340, 0.6278, 0.6249, 0.6211, 0.6188, 0.6179, 0.6147,
        0.6131, 0.6139, 0.6149, 0.6173, 0.6173, 0.6149, 0.6139, 0.6131, 0.6147,
        0.6179, 0.6188, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])
76 20 tensor([0.0000, 0.6446, 0.6340, 0.6278, 0.6249, 0.6211, 0.6188, 0.6179, 0.6147,
   

90 10 tensor([0.0000, 0.8065, 0.8060, 0.8026, 0.8008, 0.7965, 0.7952, 0.7969, 0.7978,
        0.8009, 0.8075, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000])
91 1 tensor([0.0000, 0.8184, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
    

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 12.,
        14., 15., 16.,  8.,  7.,  6.,  5.,  4.,  3.,  2.,  1., 25.,  1.,  2.,
         3.,  4.,  5.,  6., 32., 33., 34., 10., 11., 12., 12., 11., 10.,  9.,
         8.,  7.,  6.,  5.,  4.,  3.,  2.,  1., 50.,  1.,  2.,  3.,  4.,  5.,
         6.,  7.,  8.,  9., 10., 11., 12., 12., 11., 10.,  9.,  8.,  7.,  6.,
         5.,  4.,  3.,  2.,  1., 25.,  1., 23.,  3.,  4., 20.,  6.,  7., 17.,
         9., 10., 11., 13., 12., 11., 10.,  9.,  8.,  7.,  6.,  5.,  4.,  3.,
         2.,  1.])